In [ ]:
using GuSTO

In [ ]:
robot = Plane()
model = Airplane(robot)
env = Forest();

In [ ]:
using MAT
vars = matread(joinpath(Pkg.dir("GuSTO"), "src", "dynamics", "init_straight_line_sos.mat"))
# vars = matread(joinpath(Pkg.dir("GuSTO"), "src", "dynamics", "init_feasible_sos.mat"))
Xfull,Ufull,Tsfull = vars["X"], vars["U"], vars["t_ref"];

In [ ]:
N = 41
x_init = [1,1,5,0,robot.v_hat,0,0,robot.alpha_0]
goal_box = [110 110 5; 120 120 28]; mp = 0.5*(goal_box[1,:]+goal_box[2,:])
x_goal = [mp[1],mp[2],mp[3],0,robot.v_hat,0,0,robot.alpha_0]
tf_guess = Tsfull[end] # 32.3

full_idx = [round(Int,i) for i in collect(linspace(1,length(Tsfull),N))];

In [ ]:
# init_traj_straightline
X0 = repmat(x_init, 1, N)
for k in 1:3
    X0[k,:] = collect(linspace(x_init[k],x_goal[k],N))
end

# init_traj_line_sos
vars = matread(joinpath(Pkg.dir("GuSTO"), "src", "dynamics", "init_straight_line_sos.mat"))
X0 = vars["X"]
U0 = vars["U"]
Nidx = size(X0,2)
idx = [round(Int,i) for i in collect(linspace(1,Nidx,N))]
X0 = X0[:,idx]
U0 = U0[:,idx[1:end-1]]
x_goal = X0[:,end]

# init_traj_feasible_sos
vars = matread(joinpath(Pkg.dir("GuSTO"), "src", "dynamics", "init_feasible_sos.mat"))
X0 = vars["X"]
U0 = vars["U"]
Nidx = size(X0,2)
idx = [round(Int,i) for i in collect(linspace(1,Nidx,N))]
X0 = X0[:,idx]
U0 = U0[:,idx[1:end-1]]
x_goal = X0[:,end];

PD = ProblemDefinition(robot, model, env, x_init, x_goal);

In [ ]:
dt = tf_guess/(N-1)
feasible_cost = 0
for k in 1:N-1
    feasible_cost += dt * norm(U0[:,k],2)
end
feasible_cost

In [ ]:
TOPgusto = TrajectoryOptimizationProblem(PD, N, tf_guess, fixed_final_time=true)
TOSgusto = TrajectoryOptimizationSolution(TOPgusto)
solve_SCP!(TOSgusto, TOPgusto, solve_gusto_cvx!, init_traj_feasible_sos, "Gurobi");

In [ ]:
@show TOSgusto.SCPS.successful
@show TOSgusto.SCPS.converged
@show TOSgusto.SCPS.iterations
@show TOSgusto.SCPS.iter_elapsed_times
@show TOSgusto.SCPS.total_time
@show TOSgusto.SCPS.accept_solution
@show TOSgusto.SCPS.prob_status
@show TOSgusto.SCPS.convergence_measure
@show TOSgusto.SCPS.param.alg.omega_vec
@show TOSgusto.SCPS.param.alg.Delta_vec

In [ ]:
PyPlot.figure()
PyPlot.plot3D(TOSgusto.traj.X[1,:],TOSgusto.traj.X[2,:],TOSgusto.traj.X[3,:], "g-.")
PyPlot.plot3D(X0[1,:], X0[2,:], X0[3,:], "k--")

In [ ]:
TOPmao = TrajectoryOptimizationProblem(PD, N, tf_guess, fixed_final_time=true)
TOSmao = TrajectoryOptimizationSolution(TOPmao)
solve_SCP!(TOSmao, TOPmao, solve_mao_cvx!, init_traj_feasible_sos, "Gurobi")

In [ ]:
[TOSmao.traj.X[:,end] x_goal]

In [ ]:
TOSmao.SCPS.converged

In [ ]:
PyPlot.figure()
PyPlot.plot3D(TOSmao.traj.X[1,:],TOSmao.traj.X[2,:],TOSmao.traj.X[3,:], "g-.")
PyPlot.plot3D(X0[1,:], X0[2,:], X0[3,:], "k--")

In [ ]:
TOPtrajopt = TrajectoryOptimizationProblem(PD, N, tf_guess, fixed_final_time=true)
TOStrajopt = TrajectoryOptimizationSolution(TOPtrajopt)
solve_SCP!(TOStrajopt, TOPtrajopt, solve_trajopt_cvx!, init_traj_feasible_sos, "Gurobi")

In [ ]:
sum(TOStrajopt.SCPS.iter_elapsed_times)

In [ ]:
function plot_shape(rect::HyperRectangle,plot_color::String="blue")
  lenX,lenY,lenZ = rect.widths 

  (x1,y1,z1) = rect.origin
  (x2,y2,z2) = rect.origin+rect.widths
  
  xs = [x1; x2; x2; x1; x1]
  ys = [y1; y1; y2; y2; y1]
  zs = [z1; z1; z1; z1; z1]
 
  zs2 = [z2; z2; z2; z2; z2]

  xs3 = [x1; x1; x2; x2; x2; x2; x1; x1]
  ys3 = [y2; y2; y2; y2; y1; y1; y1; y1]
  zs3 = [z1; z2; z2; z1; z1; z2; z2; z1]

    # might need to change to 
    # PyPlot.mesh(xs, ys, reshape(zs,length(zs),1), color=plot_color)
    # PyPlot.mesh(xs, ys, reshape(zs2,length(zs),1), color=plot_color)
    # PyPlot.mesh(xs, ys, reshape(zs3,length(zs),1), color=plot_color)
  PyPlot.mesh(xs, ys, zs, color=plot_color)
  PyPlot.mesh(xs, ys, zs2, color=plot_color)
  PyPlot.mesh(xs3, ys3, zs3, color=plot_color)
end

In [ ]:
TOSmao.SCPS.J_true[end]
sum(TOSgusto.SCPS.iter_elapsed_times)

In [ ]:
PyPlot.figure()
#PyPlot.plot(TOSmao.SCPS.J_true[2:end], "g-.")
PyPlot.plot(TOSgusto.SCPS.J_true[2:end], "b--")
PyPlot.plot(TOStrajopt.SCPS.J_true[2:end], "g-.")

In [ ]:
sum(TOStrajopt.SCPS.iter_elapsed_times)

In [ ]:
using MAT
file = matopen("airplane_results.mat", "w")
write(file, "traj_sos", X0)
write(file, "traj_gusto", TOSgusto.traj.X)
write(file, "traj_trajopt", TOStrajopt.traj.X)
write(file, "traj_mao", TOSmao.traj.X)
close(file)

In [ ]:
PyPlot.plot(TOSgusto.SCPS.J_true[2:end])

In [ ]:
minimum(TOSgusto.traj.X[3,:])

In [ ]:
pygui(true)
for zone in env.obstacles_set_render
    plot_shape(zone, "r")
end

goal_zone = HyperRectangle(Vec3f0(model.goal_min), Vec3f0(model.goal_max-model.goal_min))
plot_shape(goal_zone, "g")

PyPlot.plot3D(TOSmao.traj.X[1,:],TOSmao.traj.X[2,:],TOSmao.traj.X[3,:], "g-.", label="Mao et al.")
PyPlot.plot3D(TOSgusto.traj.X[1,:],TOSgusto.traj.X[2,:],TOSgusto.traj.X[3,:], "b--", label="GuSTO")
PyPlot.plot3D(TOStrajopt.traj.X[1,:],TOStrajopt.traj.X[2,:],TOStrajopt.traj.X[3,:], "k", label="TrajOpt")
PyPlot.plot3D(Xfull[1,:], Xfull[2,:], Xfull[3,:], "k--", label="Seed")
PyPlot.legend(loc="lower right")
PyPlot.xlabel("x [m]"); PyPlot.ylabel("y [m]"); PyPlot.zlabel("z [m]");

#PyPlot.figure()
#PyPlot.plot(TOSmao.traj.X[1,:],TOSmao.traj.X[2,:], "g-.", label="Mao")
#PyPlot.plot(TOSgusto.traj.X[1,:],TOSgusto.traj.X[2,:], "b--", label="GuSTO")
#PyPlot.plot(TOStrajopt.traj.X[1,:],TOStrajopt.traj.X[2,:], "k", label="TrajOpt")
#PyPlot.plot(Xfull[1,:], Xfull[2,:], "k--", label="Seed")
#PyPlot.xlabel("x [m]"); PyPlot.ylabel("y [m]");

In [ ]:
@show TOSgusto.SCPS.accept_solution
@show TOSmao.SCPS.accept_solution

In [ ]:
@show TOSgusto.SCPS.J_true[end]
@show TOSmao.SCPS.J_true[end]
@show TOStrajopt.SCPS.J_true[end]

In [ ]:
env.keepout_zones

In [ ]:
vis = Visualizer()
delete!(vis)

trans = Translation(100.,100.,0.)
settransform!(vis["/Cameras/default"],trans)

vis[:obstacles]
for (idx,obs) in enumerate(env.obstacle_set)
    setobject!(vis[:obstacles][Symbol(string("obs",idx))], 
        Object(obs,MeshBasicMaterial(color=RGBA(1.0,0.,0.,0.3))))
end

vis[:goal]
setobject!(vis[:goal][:goal], 
    Object(HyperRectangle(Vec3f0(model.goal_min), Vec3f0(model.goal_max-model.goal_min)),
        MeshBasicMaterial(color=RGBA(0,1.0,0.,0.7))))

verts = Vector{Point3f0}(0)
colors = Vector{RGB{Float32}}(0)
for k in 1:size(TOSgusto.traj.X,2)
    push!(verts, Point3f0(TOSgusto.traj.X[1:3,k]))
    push!(colors, RGB(1.,0.,0.))
end
vis[:samples]
setobject!(vis[:samples][:GuSTO], PointCloud(verts,colors))

#vis[:workspace]
#for (idx,ws) in enumerate(env.keepin_zones)
#    setobject!(vis[:workspace][Symbol(string("ws",idx))],
#        Object(ws,MeshBasicMaterial(color=RGBA(0.95,0.93,0.26,0.3))))
#end

#ab = Astrobee()
#mvis = MechanismVisualizer(
#    ab.mechanism,
#    URDFVisuals(AstrobeeRobot.urdfpath(), package_path=[dirname(dirname(AstrobeeRobot.urdfpath()))]),
#    vis);

#Qs = Vector{Vector{Float64}}(0)
#for k in 1:N
#    q = [quat_inv(mrp2quat(TOSgusto.traj.X[7:9,k]));TOSgusto.traj.X[1:3,k]]
#    push!(Qs,q)
#end

#sleep(3)
#setanimation!(mvis,1:length(Qs),Qs)

plot_in_cell = false
plot_in_cell ? IJuliaCell(vis) : open(vis)